In [ ]:
import torch
import numpy as np
import torch.nn as nn
import os
import sys
sys.path.append('../')
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torchvision
from voc import *
from coco import *
import torchvision.transforms as transforms
from torchvision.models import resnet152, resnet101, resnet18, resnet34, resnet50
from tqdm import tqdm
import json
import pandas as pd
import matplotlib.pyplot as plt
from config import seed_everything
seed_everything(0)
import timm

# from models import *
from backbones.config import config
import pathlib
from torch import nn, einsum
from einops import rearrange, repeat

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = timm.create_model(config["swin"], num_classes=80, pretrained=True).to(device)
pre = torch.nn.Sequential(*[model.patch_embed,
model.pos_drop])

print(len(model.layers))
post = torch.nn.Sequential(*[model.norm,
# model.fc_norm,
model.head])

In [ ]:
inp = torch.randn(1, 3, 384, 384).to(device)

ln = nn.LayerNorm(1024).to(device)
def forward_(model, inp):
  inp = pre(inp)
  print(inp.shape)

  int_li = []
  for b in model.layers:
    inp = b(inp)
    print(inp.shape)
    int_li.append(inp)

  inp = ln(inp)
  print(inp.shape)
  inp = post(inp)
  print(inp.shape)
  return inp, int_li

inp, int_li = forward_(model, inp)

In [ ]:
ll1 = nn.Linear(2304, 1, bias=False).to(device)
ll2 = nn.Linear(576, 1, bias=False).to(device)
ll3 = nn.Linear(144, 1, bias=False).to(device)
ll4 = nn.Linear(144, 1, bias=False).to(device)

In [ ]:
def get_kv(int_li):
  res = None
  softmax = nn.Softmax(dim=1)
  val, ind = softmax(ll1(int_li[0].mT)).sort(dim=1, descending=True)
  top4 = ind[:,0:4,].flatten()
  print(top4)
  print(int_li[0][:,:,top4].shape)
  res = int_li[0][:,:,top4]

  val, ind = softmax(ll2(int_li[1].mT)).sort(dim=1, descending=True)
  top4 = ind[:,0:4,].flatten()
  print(top4)
  print(int_li[1][:,:,top4].shape)
  res = torch.cat((res, int_li[1][:,:,top4]), dim=1)

  val, ind = softmax(ll3(int_li[2].mT)).sort(dim=1, descending=True)
  top4 = ind[:,0:4,].flatten()
  print(top4)
  print(int_li[2][:,:,top4].shape)
  res = torch.cat((res, int_li[2][:,:,top4]), dim=1)

  val, ind = softmax(ll4(int_li[3].mT)).sort(dim=1, descending=True)
  top4 = ind[:,0:4,].flatten()
  print(top4)
  print(int_li[3][:,:,top4].shape)
  res = torch.cat((res, int_li[3][:,:,top4]), dim=1)

  return res

get_kv(int_li).shape
# ll2 = nn.Linear(512, 1, bias=False).to(device)
# softmax = nn.Softmax(dim=1)
# val, ind = softmax(ll2(int_li[1])).sort(dim=1, descending=True)
# ind[0,0:4]


In [ ]:
dim = 3168
inner_dim = 3168//1
group_queries = True
group_key_values = True
offset_groups = 1
heads=1
to_q = nn.Conv1d(144, inner_dim, 1, groups = offset_groups if group_queries else 1, bias = False).to(device)
to_k = nn.Conv1d(dim, inner_dim, 1, groups = offset_groups if group_key_values else 1, bias = False).to(device)
to_v = nn.Conv1d(dim, inner_dim, 1, groups = offset_groups if group_key_values else 1, bias = False).to(device)
to_out = nn.Conv1d(inner_dim, 144, 1).to(device)

In [ ]:

group = lambda t: rearrange(t, 'b (g d) n -> (b g) d n', g = offset_groups)
grouped_queries = group(q)
grouped_queries.shape #torch.Size([4, 49, 768])



In [ ]:
def attention( out, int_li):
  kv = get_kv(int_li)
  k, v = to_k(kv), to_v(kv)
  # k.shape, v.shape #(torch.Size([1, 196, 768]), torch.Size([1, 196, 768]))
  q = to_q(out)
  # q.shape #torch.Size([1, 196, 768])

  # split out heads
  q, k, v = map(lambda t: rearrange(t, 'b (h d) n -> b h n d', h = heads), (q, k, v))

  # query / key similarity
  sim = einsum('b h i d, b h j d -> b h i j', q, k)
  # numerical stability

  sim = sim - sim.amax(dim = -1, keepdim = True).detach()

  # attention
  dropout = nn.Dropout(0.0)
  attn = sim.softmax(dim = -1)
  attn = dropout(attn)
  attn.shape

  # aggregate and combine heads

  out = einsum('b h i j, b h j d -> b h i d', attn, v)
  out = rearrange(out, 'b h n d -> b (h d) n')
  out = to_out(out)
  print(out.shape)
  return out
attention(int_li[3], int_li)

# Datasets

In [ ]:
train_dataset = COCO2017('../data/coco', phase='train')
# partial=torch.utils.data.Subset(test_dataset, list(range(100)))
# train_dataset = Voc2007Classification('data/voc', 'trainval', inp_name='data/voc/voc_glove_word2vec.pkl', LT=True)
# test_dataset = Voc2007Classification('data/voc', 'test', inp_name='data/voc/voc_glove_word2vec.pkl')
# train_dataset = COCO2014('data/coco', phase='train', inp_name='data/coco/coco_glove_word2vec.pkl')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
train_dataset.transform = transforms.Compose([
                Warp(384),
                transforms.ToTensor(),
                normalize,
            ])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=50, shuffle=True)


In [ ]:
criterion = torch.nn.MultiLabelSoftMarginLoss()

In [ ]:

# model = get_model(i).to(device)
model = model.train()
act_li = np.array([[]])
for i, (input, target) in tqdm(enumerate(train_loader)):
  img, path = input
  target[target == 0] = 1
  target[target == -1] = 0
  feat_Var = torch.autograd.Variable(img).float().to(device)
  
  # output = model(feat_Var, None).detach()
  # output = model(feat_Var)
  output, int_li = forward_(model, feat_Var)
  
  out = attention(output, int_li)
  out = out.mean(dim=1)
  criterion(out, target)

